In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
np.set_printoptions(precision=3, suppress=True)

print(pd.__version__)

1.3.4


# 1. Ghép nối/trộn các dataframe

In [2]:
np.random.seed(1)
a1 = pd.DataFrame(np.random.randint(0, 10, size=(10,2)), columns=['C1', 'C2'])
a2 = pd.DataFrame(np.random.randint(0, 10, size=(5,2)), index=range(1,6), columns=['C3', 'C4'])

In [3]:
a1

,C1,C2
0,5,8
1,9,5
2,0,0
3,1,7
4,6,9
5,2,4
6,5,2
7,4,2
8,4,7
9,7,9


In [4]:
a2

,C3,C4
1,1,7
2,0,6
3,9,9
4,7,6
5,9,1


## 1.1. Sử dụng hàm `pd.concat`

**Ghép theo chiều ngang**

In [5]:
# Tham số: axis=1
pd.concat((a1,a2), axis=1)

,C1,C2,C3,C4
0,5,8,NaN,NaN
1,9,5,1.0,7.0
2,0,0,0.0,6.0
3,1,7,9.0,9.0
4,6,9,7.0,6.0
5,2,4,9.0,1.0
6,5,2,NaN,NaN
7,4,2,NaN,NaN
8,4,7,NaN,NaN
9,7,9,NaN,NaN


==> _Lưu ý: các dòng có index được ghép_

**Ghép theo chiều dọc**

(Tên cột phải giống nhau)

In [6]:
np.random.seed(2)
a3 = pd.DataFrame(np.random.randint(0, 10, size=(3,2)), columns=['C1', 'C2'])
a3

,C1,C2
0,8,8
1,6,2
2,8,7


In [80]:
# Tham số: axis=0
pd.concat([a1,a3], axis=0)

,C1,C2
0,5,8
1,9,5
2,0,0
3,1,7
4,6,9
5,2,4
6,5,2
7,4,2
8,4,7
9,7,9


In [7]:
# Phải thực hiện reset_index
pd.concat([a1,a3], axis=0).reset_index(drop=True)

,C1,C2
0,5,8
1,9,5
2,0,0
3,1,7
4,6,9
5,2,4
6,5,2
7,4,2
8,4,7
9,7,9


## 1.2. Sử dụng hàm `df.append`

**(Ghép theo chiều dọc: tên cột phải giống nhau)**

In [9]:
a1.append(a3).reset_index(drop=True)

,C1,C2
0,5,8
1,9,5
2,0,0
3,1,7
4,6,9
5,2,4
6,5,2
7,4,2
8,4,7
9,7,9


In [84]:
# Thiết lập tham số 'ignore_index' thay vì phải gọi 'reset_index'
a1.append(a3, ignore_index=True)

,C1,C2
0,5,8
1,9,5
2,0,0
3,1,7
4,6,9
5,2,4
6,5,2
7,4,2
8,4,7
9,7,9


**==> Lưu ý: vẫn trả về một DF mới**

In [86]:
# Nếu tên cột khác nhau
a1.append(a2, ignore_index=True)

,C1,C2,C3,C4
0,5.0,8.0,NaN,NaN
1,9.0,5.0,NaN,NaN
2,0.0,0.0,NaN,NaN
3,1.0,7.0,NaN,NaN
4,6.0,9.0,NaN,NaN
5,2.0,4.0,NaN,NaN
6,5.0,2.0,NaN,NaN
7,4.0,2.0,NaN,NaN
8,4.0,7.0,NaN,NaN
9,7.0,9.0,NaN,NaN


## 1.3. Sử dụng hàm `join`

(Ghép theo chiều ngang)

In [15]:
# Thử thực hiện trên DF gốc
# ==> Thực hiện ghép dựa vào các index trùng nhau
a1.join(a2)

,C1,C2,C3,C4
0,5,8,NaN,NaN
1,9,5,1.0,7.0
2,0,0,0.0,6.0
3,1,7,9.0,9.0
4,6,9,7.0,6.0
5,2,4,9.0,1.0
6,5,2,NaN,NaN
7,4,2,NaN,NaN
8,4,7,NaN,NaN
9,7,9,NaN,NaN


In [ ]:
# Nếu trùng tên cột?
a1.join(a3)

**Phải thực hiện `reset_index` nếu muốn khớp vị trí**

In [10]:
# Khi reset index, phải chọn drop = True để tránh tạo ra cột 'index'
# ==> Gây trùng tên cột

a1.reset_index(drop=True).join(a2.reset_index(drop=True))

,C1,C2,C3,C4
0,5,8,1.0,7.0
1,9,5,0.0,6.0
2,0,0,9.0,9.0
3,1,7,7.0,6.0
4,6,9,9.0,1.0
5,2,4,NaN,NaN
6,5,2,NaN,NaN
7,4,2,NaN,NaN
8,4,7,NaN,NaN
9,7,9,NaN,NaN


## 1.4. Sử dụng hàm `merge`
==> Ghép theo chiều ngang, dựa trên một cột có giá trị giống nhau
(Theo kiểu SQL)

In [88]:
left = pd.DataFrame({"loại": ["foo", "bar"], "C1": [1, 2]})
right = pd.DataFrame({"loại": ["foo", "bar", "ok"], "C2": [4, 5, 6]})

In [89]:
left

,loại,C1
0,foo,1
1,bar,2


In [90]:
right

,loại,C2
0,foo,4
1,bar,5
2,ok,6


In [91]:
# Merge với tên các cột chung được tự động chọn
pd.merge(left, right)

,loại,C1,C2
0,foo,1,4
1,bar,2,5


In [93]:
# Thực hiện merge theo tên cột tự chỉ định với tham số 'on'
pd.merge(left, right, on="loại")

,loại,C1,C2
0,foo,1,4
1,bar,2,5


# 2. Một số hàm thống kê thông dụng

## 2.1. Một số thống kê cơ bản

In [20]:
np.random.seed(1)
a = np.random.randint(20, 25, size=(20,4))
df1 = pd.DataFrame(data=a, index=range(1,21), columns=list('ABCD'), dtype='uint8')
df1.head()

,A,B,C,D
1,23,24,20,21
2,23,20,20,21
3,24,24,21,22
4,24,22,24,23
5,24,22,24,22


In [21]:
df1.tail()

,A,B,C,D
16,23,24,23,21
17,23,20,20,22
18,22,21,23,24
19,22,20,20,21
20,21,23,20,20


In [22]:
df1.describe()

,A,B,C,D
count,20.000000,20.000000,20.000000,20.000000
mean,22.100000,22.400000,21.250000,21.500000
std,1.518309,1.535544,1.332785,1.147079
min,20.000000,20.000000,20.000000,20.000000
25%,20.750000,21.000000,20.000000,21.000000
50%,22.500000,23.000000,21.000000,21.000000
75%,23.000000,24.000000,22.000000,22.000000
max,24.000000,24.000000,24.000000,24.000000


In [23]:
df1.min(axis=0)

A    20
B    20
C    20
D    20
dtype: uint8

In [24]:
df1.max(axis=0)

A    24
B    24
C    24
D    24
dtype: uint8

In [95]:
df1.mean(axis='rows')

A    22.10
B    22.40
C    21.25
D    21.50
dtype: float64

In [26]:
df1.median(axis=0)

A    22.5
B    23.0
C    21.0
D    21.0
dtype: float64

In [27]:
df1.mode(axis='rows')
# df1.mode(axis=0)

,A,B,C,D
0,23.0,24.0,20,21.0
1,NaN,NaN,21,NaN


## 2.2. Lấy ra `n` dòng lớn nhất, nhỏ nhất

**Hàm `nlargest`: số dòng theo giá trị lớn nhất của một cột nào đó**

In [28]:
df1.nlargest(5, columns=['A', 'C'])

,A,B,C,D
4,24,22,24,23
5,24,22,24,22
3,24,24,21,22
6,24,21,21,20
16,23,24,23,21


In [ ]:
# Thực hiện lại với cột B, D
???


**Hàm `nsmallest`: số dòng theo giá trị nhỏ nhất của một cột nào đó**

In [29]:
df1.nsmallest(7, columns=['B'])

,A,B,C,D
2,23,20,20,21
17,23,20,20,22
19,22,20,20,21
6,24,21,21,20
7,21,21,21,21
15,22,21,20,21
18,22,21,23,24


In [ ]:
# Thực hiện lại với cột A, C
???


# 3. Sử dụng hàm value_counts

In [31]:
df1

,A,B,C,D
1,23,24,20,21
2,23,20,20,21
3,24,24,21,22
4,24,22,24,23
5,24,22,24,22
6,24,21,21,20
7,21,21,21,21
8,20,24,21,20
9,20,23,22,21
10,20,23,21,21


In [30]:
# Khi thực hiện với cả DF
# ==> Đếm số dòng trùng nhau
df1.value_counts(ascending=False)

A   B   C   D 
23  24  20  21    2
20  23  21  21    1
23  20  20  21    1
24  22  24  23    1
            22    1
    21  21  20    1
23  24  23  21    1
        22  24    1
    20  20  22    1
22  21  23  24    1
20  23  21  22    1
22  21  20  21    1
    20  20  21    1
21  23  20  20    1
    21  21  21    1
20  24  21  22    1
            20    1
    23  22  21    1
24  24  21  22    1
dtype: int64

==> _Phần để trống là "như trên"_

**Đếm trên một cột nào đó: cột C**

In [97]:
df1['C'].value_counts()

20    7
21    7
24    2
22    2
23    2
Name: C, dtype: int64

**Sắp xếp kết quả đã đếm**

In [41]:
# Sort theo số lượng đã đếm được
df1['C'].value_counts(ascending=True, normalize=False)

24    2
22    2
23    2
20    7
21    7
Name: C, dtype: int64

In [42]:
# Để lấy kết quả dạng tần suất (xác suất): tham số normalize
df1['C'].value_counts(ascending=True, normalize=True)

24    0.10
22    0.10
23    0.10
20    0.35
21    0.35
Name: C, dtype: float64

In [43]:
# Sort theo giá trị
df1['C'].value_counts().sort_index(ascending=True)

20    7
21    7
22    2
23    2
24    2
Name: C, dtype: int64

In [39]:
# Sort theo số lượng đã đếm được và giá trị
df1['C'].value_counts().sort_index(ascending=True).sort_values(ascending=True)

22    2
23    2
24    2
20    7
21    7
Name: C, dtype: int64

**Đếm theo bins (histogram)**

In [47]:
# Đếm theo bins (dành cho dữ liệu số thực hoặc liên tục)
df1['A'].value_counts(bins=3)

(22.667, 24.0]                  10
(19.994999999999997, 21.333]     7
(21.333, 22.667]                 3
Name: A, dtype: int64

In [48]:
# Để lấy ra kết quả dạng DataFrame
df1['A'].value_counts(bins=3).to_frame()

,A
"(22.667, 24.0]",10
"(19.994999999999997, 21.333]",7
"(21.333, 22.667]",3


In [54]:
# Phải tự gán lại tên cột!!!
kq = df1['A'].value_counts(bins=3).to_frame()
kq.columns = ["Count"]
kq

,Count
"(22.667, 24.0]",10
"(19.994999999999997, 21.333]",7
"(21.333, 22.667]",3


# 4. Groupby

In [55]:
vd = pd.DataFrame({"product": ["banana", "lime", "beer", "banana", "banana", "lime"], 
                              "so luong": [1, 1, 2, 5, 7, 4]})
vd

,product,so luong
0,banana,1
1,lime,1
2,beer,2
3,banana,5
4,banana,7
5,lime,4


In [63]:
kq_groupby = vd.groupby(['product'])
kq_groupby

In [64]:
# Các groups được lưu thành dict, 
# - Mỗi key là 1 group
# - Mỗi value là 1 list các index của group đó
kq_groupby.groups

{'banana': [0, 3, 4], 'beer': [2], 'lime': [1, 5]}

In [65]:
list(kq_groupby.groups.keys())

['banana', 'beer', 'lime']

**Lấy ra một group**

In [66]:
kq_groupby.get_group("banana")

,product,so luong
0,banana,1
3,banana,5
4,banana,7


**Kết hợp với các hàm khác: `sum`, `mean`,...**

In [67]:
kq_groupby.sum()

,so luong
product,
banana,13
beer,2
lime,5


In [68]:
kq_groupby.mean()

,so luong
product,
banana,4.333333
beer,2.000000
lime,2.500000


In [69]:
kq_groupby.count()

,so luong
product,
banana,3
beer,1
lime,2


In [71]:
# Có thể group theo vài cột
df1.groupby(['A', 'B']).sum()

C   D
A  B         
20 23  64  64
   24  42  42
21 21  21  21
   23  20  20
22 20  20  21
   21  43  45
23 20  40  43
   24  85  87
24 21  21  20
   22  48  45
   24  21  22

## Ví dụ: sử dụng Groupby đếm tổng số sản phẩm của 2 chi nhánh

In [72]:
cn1 = pd.Series(['banana', 'beer', 'bread', 'milk', 'milk', 'beer'], 
                name='product')
cn1

0    banana
1      beer
2     bread
3      milk
4      milk
5      beer
Name: product, dtype: object

In [73]:
cn2 = pd.Series(['banana', 'bread', 'bread', 'bread', 'bread', 'milk', 'milk', 'milk'], 
                name='product')
cn2

0    banana
1     bread
2     bread
3     bread
4     bread
5      milk
6      milk
7      milk
Name: product, dtype: object

**Thông tin về sp của từng chi nhánh**

In [74]:
# Đếm số lần xuất hiện của sp ở cn1
cn1_count = cn1.value_counts()
cn1_count

beer      2
milk      2
banana    1
bread     1
Name: product, dtype: int64

In [75]:
# Đếm số lần xuất hiện của sp ở cn2
cn2_count = cn2.value_counts()
cn2_count

bread     4
milk      3
banana    1
Name: product, dtype: int64

**Chuyển kết quả từ dạng Series sang DF**

In [76]:
# Sử dụng hàm reset_index() và gán lại tên cột
cn1_count = cn1_count.reset_index()
cn1_count.columns = ['product', 'count']
cn1_count

,product,count
0,beer,2
1,milk,2
2,banana,1
3,bread,1


In [77]:
# Thực hiện lại với cn2
cn2_count = cn2_count.reset_index()
cn2_count.columns = ['product', 'count']
cn2_count

,product,count
0,bread,4
1,milk,3
2,banana,1


**Tổng hợp lại**

In [96]:
# Ghép theo chiều dọc bằng hàm pd.concat() or df.append()
tonghop = pd.concat([cn1_count, cn2_count], ignore_index=True)
tonghop

,product,count
0,beer,2
1,milk,2
2,banana,1
3,bread,1
4,bread,4
5,milk,3
6,banana,1


In [79]:
# Dùng groupby để đếm theo 'product' và tính tổng
tonghop.groupby(['product']).sum().reset_index()

,product,count
0,banana,2
1,beer,2
2,bread,5
3,milk,5
